In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import os
import sklearn.metrics as metrics
# 绘图防止中文出错
sns.set(font="SimHei", font_scale=1.5)
plt.rcParams["font.sans-serif"] = ["Arial Unicode MS"]
plt.rcParams["axes.unicode_minus"] = False

In [2]:
data=pd.read_excel("/workspaces/LCBIO/建模数据/data1b.xlsx")
x=data.iloc[:,2:-1]
y=data["是否血肿"]
Xtrain,Xtest,Ytrain,Ytest=train_test_split(x,y,test_size=0.2,random_state=420)

In [3]:
Xtrain=Xtrain[['original_shape_Elongation', 'original_shape_MinorAxisLength',
       'NCCT_original_firstorder_Entropy',
       'NCCT_original_firstorder_InterquartileRange',
       'NCCT_original_firstorder_RobustMeanAbsoluteDeviation']]
Xtest=Xtest[['original_shape_Elongation', 'original_shape_MinorAxisLength',
       'NCCT_original_firstorder_Entropy',
       'NCCT_original_firstorder_InterquartileRange',
       'NCCT_original_firstorder_RobustMeanAbsoluteDeviation']]

In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier as XGBC

from xgboost.sklearn import XGBClassifier
XGB = XGBClassifier(min_child_weight=0.1,max_depth=5)

XGB.fit(Xtrain,Ytrain) ##拟合模型

from sklearn.metrics import accuracy_score
train_score = accuracy_score(Ytrain,XGB.predict(Xtrain))
print('train_score',train_score)
test_score = accuracy_score(Ytest,XGB.predict(Xtest))
print('test_score',test_score)

test_predict4=XGB.predict(Xtest)

from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

conf = confusion_matrix(Ytest, test_predict4)
accuracy = (conf[0, 0] + conf[1, 1]) / (conf[0, 0] + conf[0, 1] + conf[1, 0] + conf[1, 1])
print('accuracy:',accuracy)
#accuracy.append(accuracy)
sensitivity = conf[1, 1] / (conf[1, 0] + conf[1, 1])
print('sensitivity:',sensitivity)
#特异度：
speciality = conf[0, 0] / (conf[0, 1] + conf[0, 0])
print('speciality:',speciality)
#F1分数：
F1_score = 2*conf[1, 1] / (2*conf[1, 1]+conf[1, 0] + conf[0, 1])
print('F1_score:',F1_score) 
#FDR：
FDR = conf[0, 1] / (conf[0, 1] + conf[1, 1])
print('FDR:',FDR)

score = XGB.predict_proba(Xtest)[:,1]
predict = XGB.predict(Xtest)
fpr,tpr,thres = roc_curve(Ytest,score)

print("AUC:",roc_auc_score(Ytest, score, average='macro', sample_weight=None))
print('Cross-entropy loss:',metrics.log_loss(Ytest, score))

#交叉验证
scores = cross_val_score(XGB,Xtrain,Ytrain,cv=10)
scores.mean()

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

train_score 1.0
test_score 0.9
accuracy: 0.9
sensitivity: 1.0
speciality: 0.8947368421052632
F1_score: 0.5
FDR: 0.6666666666666666
AUC: 1.0
Cross-entropy loss: 0.16301893435507253


/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

0.6125

In [5]:
data=pd.read_excel("/workspaces/LCBIO/建模数据/data1b_续.xlsx")
X=data.iloc[:,2:]

In [6]:
df1=XGB.predict_proba(X)
df2=XGB.predict(X)
df = pd.DataFrame(df1, columns=["类别 0 概率", "类别 1 概率"])
df["分类"] = df2
df.to_excel("xgb概率估计结果.xlsx")

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

In [7]:
df

,类别 0 概率,类别 1 概率,分类
0,0.999480,0.000520,0
1,0.990604,0.009396,0
2,0.010495,0.989505,1
3,0.997766,0.002234,0
4,0.022494,0.977506,1
...,...,...,...
155,0.982066,0.017934,0
156,0.585536,0.414464,0
157,0.966317,0.033683,0
158,0.894466,0.105534,0


In [1]:
from matplotlib import pyplot
from numpy import loadtxt
from xgboost import XGBClassifier
model = XGBClassifier(min_child_weight=0.1,max_depth=5)  #max_depth=13,reg_lambda=0.1,objective="binary:logistic",min_child_weight=0.0001
eval_set = [(Xtrain, Ytrain), (Xtest, Ytest)]
model.fit(Xtrain, Ytrain, eval_metric=["merror", "mlogloss"],eval_set=eval_set, verbose=True)
# make predictions for test data
y_pred = model.predict(Xtest)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Ytest, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
# retrieve performance metrics
results = model.evals_result()
epochs = len(results['validation_0']['merror'])
x_axis = range(0, epochs)
# plot log loss
fig=plt.figure(figsize=(11,7),dpi=200,facecolor=None)
ax=plt.gca()
ax.spines['bottom'].set_color('black')
ax.spines['bottom'].set_linewidth('1.0')
ax.spines['top'].set_color('black')
ax.spines['top'].set_linewidth('1.0')
ax.spines['right'].set_color('black')
ax.spines['right'].set_linewidth('1.0')
ax.spines['left'].set_color('black')
ax.spines['left'].set_linewidth('1.0')

ax.plot(x_axis, results['validation_0']['mlogloss'], label='Train')
ax.plot(x_axis, results['validation_1']['mlogloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')
#pyplot.show()
pyplot.savefig('xuexi2.jpg')


# plot classification error
fig=plt.figure(figsize=(11,7),dpi=200)
ax=plt.gca(facecolor='w')
ax.spines['bottom'].set_color('black')
ax.spines['bottom'].set_linewidth('1.0')
ax.spines['top'].set_color('black')
ax.spines['top'].set_linewidth('1.0')
ax.spines['right'].set_color('black')
ax.spines['right'].set_linewidth('1.0')
ax.spines['left'].set_color('black')
ax.spines['left'].set_linewidth('1.0')

ax.plot(x_axis, results['validation_0']['merror'], label='Train')
ax.plot(x_axis, results['validation_1']['merror'], label='Test')
ax.legend()
pyplot.ylabel('Classification Error')
pyplot.title('XGBoost Classification Error')
#pyplot.show()
pyplot.savefig('xuexi2.jpg')

ModuleNotFoundError: No module named 'matplotlib'

In [19]:
model.fit(Xtrain, Ytrain,eval_metric=["merror", "mlogloss"], eval_set=eval_set, verbose=True)

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) in

XGBoostError: [13:01:45] /workspace/src/metric/multiclass_metric.cu:35: Check failed: label_error >= 0 && label_error < static_cast<int32_t>(n_class): MultiClassEvaluation: label must be in [0, num_class), num_class=1 but found 1 in label
Stack trace:
  [bt] (0) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x1ba0be) [0x7f01d1e680be]
  [bt] (1) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x9a6286) [0x7f01d2654286]
  [bt] (2) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x4ef815) [0x7f01d219d815]
  [bt] (3) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x4efad8) [0x7f01d219dad8]
  [bt] (4) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x4c843f) [0x7f01d217643f]
  [bt] (5) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(XGBoosterEvalOneIter